In [1]:
import glob
import os
import sys
from datetime import datetime
from pathlib import Path, WindowsPath
import pandas as pd
import xarray as xr
import numpy as np
from pprint import pprint



# set baseline
base1 = 2003 #emission data is only available from 2003 onwards
base2 = 2020

# set data folder
sys.path.append(Path(os.getcwd()).joinpath('src').as_posix())
base_path = r"S:\Common workspace\ETC_DI\f01_TASK_DATA\ETC_DI_2023\WP3_data_intelligence\AP23_3_2_2_Drought_and_fire_impact_on_Carbon_emissions_and_removals\Wildfires_Annual_Update_00_22"


#################################################################
#################################################################
# select tiffs
resolution = "10km" ### change this to point to the right folder
in_path = os.path.join(base_path, 'burnt_CO2_0322_10km_0')

out_path = os.path.join(base_path, f'burntCO2_{resolution}_anom_ior_vf_051023')
if not os.path.exists(out_path):
    os.mkdir(out_path)
#################################################################
#################################################################

print(datetime.now(), " loading data")

# load raster data into xarray
tif_list = [os.path.basename(f) for f in glob.glob(in_path + '/co2fire*.tif')]

# Create variable used for time axis
time_var = xr.Variable('time', pd.to_datetime([f'{fname[7:11]}-01-01' for fname in tif_list]))

# iterate for outlier removal 
for it in range(4):

    # Load in and concatenate all individual GeoTIFFs
    co2_ds= xr.concat([xr.open_dataset(os.path.join(in_path, i), engine='rasterio') for i in tif_list], dim=time_var)
    # Rename the variable to a more useful name
    co2_ds = co2_ds.rename({'band_data': 'burntCO2'})
    now = datetime.now()
    print(now, " finished loading data - iteration {it}")

    # assign coordinates (layer names) to statistics dimension
    # prepare DataArrays for saving to disk
    co2_ds = co2_ds.squeeze(dim="band")

    # extract long term stats (mean and std) for the baseline period
    baseline = co2_ds.sel(time=slice(datetime(base1,1,1), datetime(base2,1,1)))
    mean = baseline.mean(dim='time')
    std = baseline.std(dim='time', skipna=True, ddof=1)
    

    if it !=3:
        print ('Iteration {}'.format(it))
        
        temp_folder = os.path.join(out_path, f'iteration_{it}')
        if not os.path.exists(temp_folder):
            os.mkdir(temp_folder)

        # compute limits (upper and lower)
        cond_low = mean - 2*std
        cond_up = mean + 2*std  

        # select values within the range
        itr = co2_ds.where((co2_ds  > cond_low) & (co2_ds  < cond_up))
        # set nan values to -999
        itr = xr.where(np.isnan(itr), -999, itr) 
        itr['burntCO2'].rio.write_nodata(-999, inplace=True)
        itr.rio.write_crs("epsg:3035", inplace=True)

        # rename layers 
        mean = mean.rename({"burntCO2": "burntCO2_mean"})
        std = std.rename({"burntCO2": "burntCO2_std"})

        # save to disk
        mean['burntCO2_mean'].rio.to_raster(os.path.join(temp_folder, f'burntCO2_{resolution}_mean.tif'), compress='LZW')
        std['burntCO2_std'].rio.to_raster(os.path.join(temp_folder,f'burntCO2_{resolution}_std.tif'), compress='LZW')
        
        for yr in range(base1, 2022+1):
            itr['burntCO2'].sel(time=datetime(yr, 1, 1)).rio.to_raster(os.path.join(temp_folder,
                               f'burntCO2_{resolution}_{yr}.tif'), compress='LZW')

        tif_list = [os.path.basename(f) for f in glob.glob(os.path.join(temp_folder, '*.tif'))]
        tif_list = tif_list[:-2]
        in_path = temp_folder


    else:
        print ('Final Iteration {}'.format(it))

        # compute z-score
        deviation = co2_ds - mean
        anom = deviation / std

        # set nan values to -999
        anom = xr.where(np.isnan(anom), -999, anom)
        anom = xr.where(np.isfinite(anom), anom, -999) # to remove infinite values
        anom['burntCO2'].rio.write_nodata(-999, inplace=True)
        anom.rio.write_crs("epsg:3035", inplace=True)

        deviation = xr.where(np.isnan(deviation), -999, deviation) 
        deviation['burntCO2'].rio.write_nodata(-999, inplace=True)
        deviation.rio.write_crs("epsg:3035", inplace=True)

        # count all values for final computation
        count = co2_ds.count("time")

        # rename layers 
        mean = mean.rename({"burntCO2": "burntCO2_mean"})
        std = std.rename({"burntCO2": "burntCO2_std"})
        anom = anom.rename({"burntCO2": "burntCO2_anom"})
        deviation = deviation.rename({"burntCO2": "burntCO2_deviation"})
        count = count.rename({"burntCO2": "ior_count"})

        # save to disk
        mean['burntCO2_mean'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_mean.tif'), compress='LZW')
        std['burntCO2_std'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_std.tif'), compress='LZW')
        count['ior_count'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_count.tif'), compress='LZW')

        for yr in range(base1, 2022+1):
            anom['burntCO2_anom'].sel(time=datetime(yr, 1, 1)).rio.to_raster(os.path.join(out_path,
                                                                                    f'burntCO2_{resolution}_anom_{yr}.tif'), compress='LZW')
            deviation['burntCO2_deviation'].sel(time=datetime(yr, 1, 1)).rio.to_raster(os.path.join(out_path,
                                                                                    f'burntCO2_{resolution}_deviation_{yr}.tif'), compress='LZW')

2023-10-18 12:56:18.138984  loading data
2023-10-18 12:56:20.731995  finished loading data - iteration {it}
Iteration 0
2023-10-18 12:56:26.918999  finished loading data - iteration {it}


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Iteration 1
2023-10-18 12:56:32.231033  finished loading data - iteration {it}


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Iteration 2
2023-10-18 12:56:37.316030  finished loading data - iteration {it}


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Final Iteration 3


"        # save to disk\n        mean['burntCO2_mean'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_mean.tif'), compress='LZW')\n        std['burntCO2_std'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_std.tif'), compress='LZW')\n        count['ior_count'].rio.to_raster(os.path.join(out_path, f'burntCO2_{resolution}_count.tif'), compress='LZW')\n\n        for yr in range(base1, 2022+1):\n            anom['burntCO2_anom'].sel(time=datetime(yr, 1, 1)).rio.to_raster(os.path.join(out_path,\n                                                                                    f'burntCO2_{resolution}_anom_{yr}.tif'), compress='LZW')\n            deviation['burntCO2_deviation'].sel(time=datetime(yr, 1, 1)).rio.to_raster(os.path.join(out_path,\n                                                                                    f'burntCO2_{resolution}_deviation_{yr}.tif'), compress='LZW')"